# Input 

Detailed overview on the input processor and all its functionalities


### 0. Loading packages & settings



In [1]:
import valenspy as vp

## Loading inputdata in valenspy

Input convertor

Input manager 

input variables: CORDEX naming of variables + link to overview file

## 1. Input convertor